<a href="https://colab.research.google.com/github/DhruvTomar-ai/Rental/blob/master/AutoFormFiller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries for Google Colab
!pip install selenium pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 15.1 MB/s eta 0:00:00


In [2]:
# Install Chrome and ChromeDriver in Colab
!apt-get update # to update ubuntu's package list
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,244 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [77.5 kB]
Get:13 https://developer.download.nvidia.com/compute/cud

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from google.colab import files

# Upload and load the Excel file
print("Uploading Excel file...")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
data = pd.read_excel(filename)

# Clean column names: strip spaces and convert to lowercase
data.columns = data.columns.str.strip().str.lower()
print("Column names after cleaning:", list(data.columns))

# Kill any existing chromedriver (Colab-specific)
print("Killing any existing ChromeDriver processes...")
!killall chromedriver

# Chrome options for Colab
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Google Form URL
form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSeBOeKebzwkrQwLOpL3oAWOVb1YFJ_EaRvZSR0bsrp6CvfUAA/viewform'

print(f"Starting form submission for {len(data)} entries...\n")

# Loop through each row in the Excel sheet
for index, row in data.iterrows():
    print(f"\nOpening form for row {index + 1}: {row.get('first name', 'N/A')} {row.get('last name', 'N/A')}")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(form_url)
    time.sleep(0.2)

    try:
        # Extract values
        id_value = str(row.get('id', ''))
        first_name_value = str(row.get('first name', ''))
        middle_initial_value = str(row.get('middel initial', ''))
        last_name_value = str(row.get('last name', ''))
        father_name_value = str(row.get("fathers name", ''))
        dob_raw = row.get('dob', '')  # Expecting datetime or string

        dob_value = ''
        if pd.notnull(dob_raw):
            dob_value = pd.to_datetime(dob_raw).strftime('%Y-%m-%d')

        # Fill the form
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[aria-labelledby="i1 i4"]'))).send_keys(id_value)
        print(" - ID entered")

        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[aria-labelledby="i6 i9"]'))).send_keys(first_name_value)
        print(" - First Name entered")

        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[aria-labelledby="i11 i14"]'))).send_keys(middle_initial_value)
        print(" - Middle Initial entered")

        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[aria-labelledby="i16 i19"]'))).send_keys(last_name_value)
        print(" - Last Name entered")

        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[aria-labelledby="i21 i24"]'))).send_keys(father_name_value)
        print(" - Father's Name entered")

        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="date"]'))).send_keys(dob_value)
        print(f" - DOB entered: {dob_value}")

        # Submit the form
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//span[text()="Submit"]'))).click()
        print(f"[{index + 1}] ✅ Form submitted for {first_name_value} {last_name_value}")

        time.sleep(0.2)

    except Exception as e:
        print(f"❌ Error on row {index + 1}: {e}")

    finally:
        driver.quit()


Uploading Excel file...


Saving Book1.xlsx to Book1.xlsx
Column names after cleaning: ['id', 'first name', 'middel initial', 'last name', 'fathers name', 'dob']
Killing any existing ChromeDriver processes...
chromedriver: no process found
Starting form submission for 5 entries...


Opening form for row 1: reyes stought
 - ID entered
 - First Name entered
 - Middle Initial entered
 - Last Name entered
 - Father's Name entered
 - DOB entered: 1983-02-01
[1] ✅ Form submitted for reyes stought

Opening form for row 2: ron will
 - ID entered
 - First Name entered
 - Middle Initial entered
 - Last Name entered
 - Father's Name entered
 - DOB entered: 1988-04-01
[2] ✅ Form submitted for ron will

Opening form for row 3: cavin cane
 - ID entered
 - First Name entered
 - Middle Initial entered
 - Last Name entered
 - Father's Name entered
 - DOB entered: 1989-05-03
[3] ✅ Form submitted for cavin cane

Opening form for row 4: call woop
 - ID entered
 - First Name entered
 - Middle Initial entered
 - Last Name entered
 -

In [ ]:
!pip freeze > requirements.txt


In [ ]:
from google.colab import files
files.download('requirements.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
#same logic just by using XPATH this time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from google.colab import files

# Upload Excel
print("Uploading Excel file...")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
data = pd.read_excel(filename)

# Clean column names
data.columns = data.columns.str.strip().str.lower()
print("Column names after cleaning:", list(data.columns))

# Kill any existing ChromeDriver (for Colab)
print("Killing any existing ChromeDriver processes...")
!killall chromedriver

# Setup Chrome options for headless mode (Colab safe)
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Google Form URL
form_url = 'https://forms.gle/yCTiiCBhdkMoYSH9A'

print(f"Starting form submission for {len(data)} entries...\n")

# Start submission loop
for index, row in data.iterrows():
    print(f"\nOpening form for row {index + 1}: {row.get('first name', 'N/A')} {row.get('last name', 'N/A')}")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(form_url)
    time.sleep(2)

    try:
        # Get values from Excel
        id_value = str(row.get('id', ''))
        first_name_value = str(row.get('first name', ''))
        middle_initial_value = str(row.get('middel initial', ''))
        last_name_value = str(row.get('last name', ''))
        father_name_value = str(row.get("fathers name", ''))
        dob_raw = row.get('dob', '')

        dob_value = ''
        if pd.notnull(dob_raw):
            dob_value = pd.to_datetime(dob_raw).strftime('%Y-%m-%d')

        wait = WebDriverWait(driver, 10)

        # Fill each field using XPATH
        wait.until(EC.element_to_be_clickable((By.XPATH, '(//input[@type="text"])[1]'))).send_keys(id_value)
        print(" - ID entered")

        wait.until(EC.element_to_be_clickable((By.XPATH, '(//input[@type="text"])[2]'))).send_keys(first_name_value)
        print(" - First Name entered")

        wait.until(EC.element_to_be_clickable((By.XPATH, '(//input[@type="text"])[3]'))).send_keys(middle_initial_value)
        print(" - Middle Initial entered")

        wait.until(EC.element_to_be_clickable((By.XPATH, '(//input[@type="text"])[4]'))).send_keys(last_name_value)
        print(" - Last Name entered")

        wait.until(EC.element_to_be_clickable((By.XPATH, '(//input[@type="text"])[5]'))).send_keys(father_name_value)
        print(" - Father’s Name entered")

        wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@type="date"]'))).send_keys(dob_value)
        print(f" - DOB entered: {dob_value}")

        # Submit button
        wait.until(EC.element_to_be_clickable((By.XPATH, '//span[text()="Submit"]'))).click()
        print(f"[{index + 1}] ✅ Form submitted for {first_name_value} {last_name_value}")
        time.sleep(2)

    except Exception as e:
        print(f"❌ Error on row {index + 1}: {e}")

    finally:
        driver.quit()


Uploading Excel file...


KeyboardInterrupt: 